I haven't done this in a while. It used to be Yahoo Finance API, then Quantopian and Quandl, I'm way behind. I hear good things about _Alpha Vantage_.

In [11]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from functools import lru_cache

load_dotenv()

@lru_cache(maxsize=None)
def fetch_dailies(ticker):
  url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&outputsize=full&symbol={ticker}&apikey={os.getenv('ALPHAVANTAGE_API_KEY')}"
  response = requests.get(url)
  response = response.json()
  # Convert the response to a pandas DataFrame
  df = pd.DataFrame(response['Time Series (Daily)']).T
  df.index = pd.to_datetime(df.index)
  df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
  df['Ticker'] = ticker
  # Convert 'Open' and 'Close' columns to numeric
  df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
  df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

  # Calculate the overnight return
  df['Overnight_Return'] = (df['Open'] / df['Close'].shift(-1)) - 1

  return df


In [12]:
fetch_dailies('AAPL')

,Open,High,Low,Close,Volume,Ticker,Overnight_Return
2024-02-28,182.51,183.1200,180.1300,181.42,48953939,AAPL,-0.000657
2024-02-27,181.10,183.9225,179.5600,182.63,54318851,AAPL,-0.000331
2024-02-26,182.24,182.7600,180.6500,181.16,40867421,AAPL,-0.001534
2024-02-23,185.01,185.0400,182.2300,182.52,45119677,AAPL,0.003471
2024-02-22,183.48,184.9550,182.4600,184.37,52292208,AAPL,0.006362
...,...,...,...,...,...,...,...
1999-11-05,84.62,88.3700,84.0000,88.31,3721500,AAPL,0.011959
1999-11-04,82.06,85.3700,80.6200,83.62,3384700,AAPL,0.006871
1999-11-03,81.62,83.2500,81.0000,81.50,2932700,AAPL,0.017072
1999-11-02,78.00,81.6900,77.3100,80.25,3564600,AAPL,0.004896


Easy, awesome, I'm game.